In [1]:
import numpy as np
import os
import pandas as pd
import time
from itertools import product
import math

In [2]:
start = 165900

end = 221165


In [3]:
df_disease_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_whole.tsv")
df_drug_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_indep_drug_similarity_whole.tsv")
asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association_indep.tsv")

print len(asso)
pos = asso[asso.association ==1]

new_asso_li = list()
for index, row in asso.iterrows():
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    new_asso_li.append((query_drug, query_disease, query_asso))

new_pos_li = list()
for index2, row2 in pos.iterrows():
    pos_drug = row2['drug']
    pos_disease = row2['disease']
    new_pos_li.append((pos_drug, pos_disease))

    
print 'start : ' + time.strftime('%a %H:%M:%S')
desc_pair = list(product(new_asso_li, new_pos_li))
print 'end : ' + time.strftime('%a %H:%M:%S')
print len(desc_pair)


descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
for pair in desc_pair[start:]:
    query_drug = pair[0][0]
    query_disease = pair[0][1]
    query_asso = pair[0][2]
    pos_drug = pair[1][0]
    pos_disease = pair[1][1]
    if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
    else:
        found_drug = df_drug_sim[(df_drug_sim.DrugBank_ID1 == query_drug) & (df_drug_sim.DrugBank_ID2 == pos_drug)]
        drug_cheSim = found_drug['CheSim'].values[0]
        drug_pheSim = found_drug['PheSim'].values[0]
    
        found_disease = df_disease_sim[(df_disease_sim.Disease_ID1 == query_disease) & (df_disease_sim.Disease_ID2 == pos_disease)]
        disease_pheSim = found_disease['PheSim'].values[0]
        disease_geSim = found_disease['GeSim'].values[0]
    
        drug_sim = [drug_cheSim, drug_pheSim]
        disease_sim = [disease_geSim, disease_pheSim]
        
        sim = [math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))]
        temp_df = pd.DataFrame(data = [[query_drug, query_disease, query_asso, sim[0], sim[1], sim[2], sim[3]]],
                           columns=['drug', 'disease', 'association', 'DrChe_DiGe', 'DrChe_DiPhe', 'DrPhe_DiGe', 'DrPhe_DiPhe'])
        descriptor = descriptor.append(temp_df)
        cnt = cnt +1
        if cnt % 10000 == 0:
            print "{} : ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')
print cnt
print descriptor.shape

descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/indep_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)

#real_desc = descriptor.groupby(['drug','disease']).max()
#real_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/indep_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t')

2485
start : Tue 17:00:39
end : Tue 17:00:39
221165
start : Tue 17:00:39
10000 : Tue 17:02:03
20000 : Tue 17:03:11
30000 : Tue 17:04:30
40000 : Tue 17:06:08
50000 : Tue 17:08:10
end : Tue 17:08:58
55228
(55228, 7)


In [6]:
descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/indep_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)